In [7]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# one-hot 
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

batch_size = 100

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

# model
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
pred = tf.nn.softmax(tf.matmul(x, W)+b)

# cost
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), 1)) # 1代表按行计算

# 梯度下降
optimmizer = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

# 初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(25):
        avag_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optimmizer, {x:batch_xs, y:batch_ys})
            avag_cost += sess.run(cost, {x:batch_xs, y:batch_ys})/total_batch
            
        if((epoch + 1) %5 == 0):
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avag_cost))
        
    print("Finished")
    correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    print("正确率:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})) 

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0005 cost= 0.463781696
Epoch: 0010 cost= 0.390997229
Epoch: 0015 cost= 0.361361458
Epoch: 0020 cost= 0.344190266
Epoch: 0025 cost= 0.332450127
Finished
正确率: 0.914


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev = 0.01)) # stddev为标准差

def model(X, w_h, w_o):
    h = tf.nn.sigmoid(tf.matmul(X, w_h))
    return tf.matmul(h, w_o)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

X = tf.placeholder("float", [None, 784])
Y = tf.placeholder("float", [None, 10])

w_h = init_weights([784, 625])
w_o = init_weights([625, 10])
py_x = model(X, w_h, w_o)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)
predict_op = tf.argmax(py_x, 1)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for i in range(10):
        for start,end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict = {X: trX[start:end], Y:trY[start:end]})
        print(i, np.mean(np.argmax(teY, axis = 1) == sess.run(predict_op, feed_dict={X:teX})))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0 0.6762
1 0.8219
2 0.8607
3 0.8799
4 0.888
5 0.8938
6 0.8972
7 0.9004
8 0.9036
9 0.9054


In [6]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

batch_size = 128
test_size = 256

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev = 0.01))

def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    # input:卷积的输入图像 filter: CNN中的卷积核 strids：卷积在图像每一维的步长
    # padding：SAME或VALID，决定卷积方式
    l1a = tf.nn.relu(tf.nn.conv2d(X, w, strides = [1,1,1,1], padding = 'SAME'))
    l1 = tf.nn.max_pool(l1a, ksize = [1,2,2,1], strides=[1,2,2,1], padding = 'SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)
    
    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2, strides=[1,1,1,1], padding="SAME"))
    l2 = tf.nn.max_pool(l2a, ksize = [1,2,2,1], strides=[1,2,2,1], padding="SAME")
    l2 = tf.nn.dropout(l2, p_keep_conv)
    
    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3, strides=[1, 1, 1, 1], padding="SAME"))
    l3 = tf.nn.max_pool(l3a, ksize = [1, 2, 2,1], strides =[1,2,2,1], padding="SAME")
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])
    l3 = tf.nn.dropout(l3, p_keep_conv)
    
    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_conv)
    
    pyx = tf.matmul(l4, w_o)
    return pyx

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28, 1)  # 28x28x1 input img
teX = teX.reshape(-1, 28, 28, 1)  # 28x28x1 input img

X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])

w = init_weights([3, 3, 1, 32])
w2 = init_weights([3, 3, 32, 64])
w3 = init_weights([3, 3, 64, 128])
w4 = init_weights([128 * 4 * 4, 625])
w_o = init_weights([625, 10])

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.arg_max(py_x, 1)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for i in range(10):
        training_batch = zip(range(0, len(trX), batch_size),
                            range(batch_size, len(trX)+1, batch_size))
        for start, end in training_batch:
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_conv: 0.8, p_keep_hidden: 0.5})
            
        test_indices = np.arange(len(teX))
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]
        
        print(i, np.mean(np.argmax(teY[test_indices], axis=1) ==
              sess.run(predict_op, feed_dict={X: teX[test_indices],
                                                         Y: teY[test_indices],
                                                         p_keep_conv: 1.0,
                                                         p_keep_hidden: 1.0})))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `argmax` instead
0 0.953125
1 0.98828125
2 0.99609375
3 0.9609375
4 0.9765625
5 0.98828125
6 0.98828125
7 0.9921875
8 0.9921875
9 0.98828125
